In [ ]:
!git clone https://github.com/praveen-jalaja/streamlit-hatefulmemedection.git

fatal: destination path 'streamlit-hatefulmemedection' already exists and is not an empty directory.


In [ ]:
!pip install tensorflow_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 589.8/589.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 58.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 32.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 15.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: h5py
    Found existing installation: h5py 3.9.0
    Uninstalling h5py-3.9.0:
      Successfully uninstalled h5py-3.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      S

In [ ]:
!pip install tensorflow_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 60.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 71.8 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.2.0
    Uninstalling keras-3.2.0:
      Successfully uninstalled keras-3.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.16.1
    Uninstalling tensorflow-2.16.1:
      Successfully uninstalled tensorflow-2.16.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-text 2.16.1 requires tensorflow<2.17,>=2.16.1; platform_machine != "arm64" or pl

In [ ]:
from PIL import Image
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras.preprocessing.image import save_img
import tensorflow_text as text
import shutil
import numpy as np
import os
import json
import sys
import random
from bs4 import BeautifulSoup
import re
from functools import partial

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'register_load_context_function'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
# Use the actual path to your file if it's located in a specific directory within Colab
file_path = 'label.txt'
# Reading the file into a DataFrame, assuming tab-delimited data
df = pd.read_csv(file_path, delimiter='\t', header=None, names=['Image', 'Label'])
# Display the first few rows to confirm it's read correctly
print(df.head())


In [ ]:
# Convert 'Label' to numeric values: 1 for 'hate', 0 for 'non_hate'
df['NumericLabel'] = df['Label'].apply(lambda x: 1 if x == 'hate' else 0)

# Convert the DataFrame to a dictionary with 'Image' as keys and 'NumericLabel' as values
image_label_dict = pd.Series(df.NumericLabel.values,index=df.Image).to_dict()

# Display the dictionary
print(image_label_dict)


In [ ]:
df = pd.read_csv('/content/csv - Sheet1.csv')

In [ ]:
df.head()

In [ ]:
text_label_dict = pd.Series(df.Text.values,index=df.Image).to_dict()
print(text_label_dict)

In [ ]:

data_dir = "/content/streamlit-hatefulmemedection/"
SLANG_PATH = "/content/streamlit-hatefulmemedection/static/slang.txt"


In [ ]:





trainable = False
max_seq_length = 128
units = 512

embedding_dim = 768
BATCH_SIZE = 50
BUFFER_SIZE = 200
attention_features_shape = 64


#image_shape = image.shape
def decontracted(phrase):
    # specific
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


with open(SLANG_PATH) as file:
    slang_map = dict(map(str.strip, line.partition('\t')[::2])
    for line in file if line.strip())

slang_words = sorted(slang_map, key=len, reverse=True)
regex = re.compile(r"\b({})\b".format("|".join(map(re.escape, slang_words))))
replaceSlang = partial(regex.sub, lambda m: slang_map[m.group(1)])


def text_preprocessing(final):
  preprocessed_text = []
  for sentance in final:
      sentance = BeautifulSoup(sentance, 'lxml').get_text()
      sentance = replaceSlang(sentance)
      sentance = decontracted(sentance)
      sentance = re.sub("\S*\d\S*", "", sentance).strip()
      sentance = re.sub('[^A-Za-z]+', ' ', sentance)
      preprocessed_text.append(sentance.strip())
  return preprocessed_text



def load_model():
  image_model = tf.keras.applications.ResNet152V2(include_top=False,
                                                weights='imagenet')
  new_input = image_model.input
  hidden_layer = image_model.layers[-1].output
  image_features_extract_model = tf.keras.Model(new_input, hidden_layer, name = "image_feature_extractor")


  tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"

  tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-10_H-768_A-12/1"

  text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text_input')
  preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing_text_layer')
  encoder_inputs = preprocessing_layer(text_input)
  encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
  outputs = encoder(encoder_inputs)

  text_features_extract_model= tf.keras.Model([text_input], [outputs['sequence_output'],outputs['pooled_output']], name = 'text_feature_extractor')

  class CNN_Encoder(tf.keras.Model):
      def __init__(self, embedding_dim=768):
          super(CNN_Encoder, self).__init__()
          self.fc = tf.keras.layers.Dense(units = embedding_dim,kernel_initializer='glorot_uniform',use_bias=False)

      def call(self, x):
          x = self.fc(x)
          x = tf.nn.relu(x)
          return x


  class BahdanauAttention(tf.keras.Model):
    def __init__(self, units):
      super(BahdanauAttention, self).__init__()
      self.W1 = tf.keras.layers.Dense(units)
      self.W2 = tf.keras.layers.Dense(units)
      self.V = tf.keras.layers.Dense(1)

    def call(self, features, hidden):
      hidden_with_time_axis = tf.expand_dims(hidden, 1)

      attention_hidden_layer = (tf.nn.tanh(self.W1(features) +
                                          self.W2(hidden_with_time_axis)))

      score = self.V(attention_hidden_layer)
      attention_weights = tf.nn.softmax(score, axis=1)
      context_vector = attention_weights * features
      context_vector = tf.reduce_sum(context_vector, axis=1)

      return context_vector, attention_weights

  class RNN_Decoder(tf.keras.Model):
    def __init__(self,  units,class_size = 2):
      super(RNN_Decoder, self).__init__()
      self.units = units

      self.gru = tf.keras.layers.GRU(self.units,
                                    return_sequences=True,
                                    return_state=True,
                                    recurrent_initializer='glorot_uniform')


      self.fc1 = tf.keras.layers.Dense(self.units)

      self.fc2 = tf.keras.layers.Dense(64, activation='relu', name='Dense_3_layer')

      self.dropout  = tf.keras.layers.Dropout(0.3)

      self.fc3  = tf.keras.layers.Dense(class_size, activation='softmax', name='classifier')

      self.attention = BahdanauAttention(self.units)

    def call(self, x, features, hidden):
      context_vector, attention_weights = self.attention(features, hidden)

      a=tf.expand_dims(context_vector, 1)

      x = tf.concat([a, x], axis=-1)


      #output, state = self.gru(x)
      output, state = self.gru(x)
      #print(output.shape,state.shape)

      x = self.fc1(output)
      x = tf.reshape(x, (-1, x.shape[2]))
      x = self.fc2(x)
      x = self.dropout(x)
      x = self.fc3(x)

      return x, state, attention_weights

    def reset_state(self, batch_size):
      return tf.zeros((batch_size, self.units))
      #return tf.zeros((batch_size, 512))


  encoder = CNN_Encoder(embedding_dim)
  decoder = RNN_Decoder( units,class_size = 2)
  optimizer = tf.keras.optimizers.Adam()

  checkpoint_path = "/content/streamlit-hatefulmemedection/checkpoints/train"
  ckpt = tf.train.Checkpoint(encoder=encoder,
                            decoder=decoder,
                            optimizer = optimizer)

  ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)
  status = ckpt.restore(ckpt_manager.latest_checkpoint)
  status.expect_partial()  # Suppresses warning for unrestored values
  start_epoch = 0
  if ckpt_manager.latest_checkpoint:
    start_epoch = int(ckpt_manager.latest_checkpoint.split('-')[-1])
    # restoring the latest checkpoint in checkpoint_path
    ckpt.restore(ckpt_manager.latest_checkpoint)

  return image_features_extract_model, text_features_extract_model, encoder, decoder



def load_image(image_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.image.resize(img, (256, 256))
    img = tf.keras.applications.resnet_v2.preprocess_input(img)
    return img


class_values = ["non-hateful","hateful"]
image_features_extract_model, text_features_extract_model, encoder, decoder = load_model()
def evaluate(image,text):

    text = text_preprocessing([text])[0]

    attention_plot = np.zeros((1, attention_features_shape))
    hidden = decoder.reset_state(batch_size=1)
    img_tensor = tf.expand_dims(load_image(image), 0)
    img_tensor = image_features_extract_model(img_tensor)
    img_tensor = tf.reshape(img_tensor,
             (img_tensor.shape[0], -1, img_tensor.shape[3]))
    text = tf.convert_to_tensor(text)
    text = tf.expand_dims(text, 0)
    text_features_seq,pooled_features = text_features_extract_model(tf.convert_to_tensor(text))

    dec_input = tf.expand_dims(pooled_features, axis = 1)

    features = encoder(img_tensor)
    predictions, hidden, attention_weights = decoder(dec_input,features, hidden)

    predicted_id = np.argmax(predictions, axis = 1).tolist()
    return predictions,class_values[predicted_id[0]]







In [ ]:

def generate_result(prediction):

  if prediction == "hateful":
    return 0

  else:
    return 1


In [ ]:
pred, predicted_class = evaluate('/content/drive/MyDrive/trainlist/image108.jpg',text_label_dict['image108.jpg'])
ans=generate_result(predicted_class)

In [ ]:
files = os.listdir('/content/drive/MyDrive/trainlist')
# Initialize counters
TP = TN = FP = FN = 0

for i in files:
    image_path = "/content/drive/MyDrive/trainlist/"+i
    pred, predicted_class = evaluate(image_path, text_label_dict[i])
    if(predicted_class=='hateful'):
      ans=1
    else:
      ans=0
    lab = image_label_dict[i]
    print("pred: ", ans, "target: ", lab)

    # Increment counters based on predictions and actual labels
    if ans == lab:
        if ans:  # True positive
            TP += 1
        else:    # True negative
            TN += 1
    else:
        if ans:  # False positive
            FP += 1
        else:    # False negative
            FN += 1

# Calculate metrics
accuracy = (TP + TN) / (TP + TN + FP + FN)
precision = TP / (TP + FP) if (TP + FP) > 0 else 0
recall = TP / (TP + FN) if (TP + FN) > 0 else 0

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")



